In [ ]:
# For manipulating arrays and DataFrames
import numpy as np
import pandas as pd

# For visualizing performance
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# For instantiating a Dense layer and sequential model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Input, Conv1D, MaxPooling1D, Flatten, Add, ReLU, LSTM, Reshape, Concatenate
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Precision, Recall
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.regularizers import l1_l2


from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split

from imblearn.over_sampling import SMOTE

import pickle
import os
from contextlib import redirect_stdout

In [ ]:
#import PTB data
X_ptb_train = pd.read_csv('data/processed/PTB/X_ptb_train.csv')
y_ptb_train = pd.read_csv('data/processed/PTB/y_ptb_train.csv')

X_ptb_train_sm = pd.read_csv('data/processed/PTB/X_ptb_train_sm.csv')
y_ptb_train_sm = pd.read_csv('data/processed/PTB/y_ptb_train_sm.csv')

X_ptb_val = pd.read_csv('data/processed/PTB/X_ptb_val.csv')
y_ptb_val = pd.read_csv('data/processed/PTB/y_ptb_val.csv')

X_ptb_test = pd.read_csv('data/processed/PTB/X_ptb_test.csv')
y_ptb_test = pd.read_csv('data/processed/PTB/y_ptb_test.csv')


display(X_ptb_train.shape)
display(y_ptb_train.shape)

display(X_ptb_train_sm.shape)
display(y_ptb_train_sm.shape)

display(X_ptb_val.shape)
display(y_ptb_val.shape)

display(X_ptb_test.shape)
display(y_ptb_test.shape)



# Reshape the data for 1D CNN
X_ptb_train_cnn = np.expand_dims(X_ptb_train, axis=2)
X_ptb_train_sm_cnn = np.expand_dims(X_ptb_train_sm, axis=2)
X_ptb_val_cnn = np.expand_dims(X_ptb_val, axis=2)
X_ptb_test_cnn = np.expand_dims(X_ptb_test, axis=2)

display(X_ptb_train_cnn.shape)
display(y_ptb_train.shape)

display(X_ptb_train_sm_cnn.shape)
display(y_ptb_train_sm.shape)

display(X_ptb_val_cnn.shape)
display(y_ptb_val.shape)

display(X_ptb_test_cnn.shape)
display(y_ptb_test.shape)

In [ ]:
#Load model
model_trained = load_model('best_dl_model/cnn6_sm_lr_bs128_epoch_06_valloss_0.0677.keras')

model_trained.summary()


# Extract features from the last convolutional block (pool_5)
feature_extractor = Model(inputs=model_trained.input, outputs=model_trained.get_layer('max_pooling1d_4').output)


feature_extractor.summary()

In [ ]:
#Freeze all convolutional layers in feature_extractor
for layer in feature_extractor.layers:
    layer.trainable = False

In [ ]:
# Build new classifier for 2 class problem
# Input same shape as original
input_layer = Input(shape=(187, 1))
x = feature_extractor(input_layer, training=False)  # Freeze convolutional base

x = Flatten()(x)
x = Dense(32, activation='relu')(x)
x = Dense(32, activation='relu')(x)
output_layer = Dense(2, activation='softmax')(x)

transfer_model = Model(inputs=input_layer, outputs=output_layer)

In [ ]:
# Learning rate with exponential decay
initial_learning_rate = 0.001
lr_schedule = ExponentialDecay(
    initial_learning_rate,
    decay_steps=10000,
    decay_rate=0.75,
    staircase=True
)

# Adam optimizer with specified hyperparameters
optimizer = Adam(
    learning_rate=lr_schedule,
    beta_1=0.9,
    beta_2=0.999
)


#Compile model
transfer_model.compile(
    optimizer=optimizer, 
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)


# Define where and how to save the best model
checkpoint = ModelCheckpoint(
    filepath='transfer/cnn6_transfer_lr_bs128_epoch_{epoch:02d}_valloss_{val_loss:.4f}.keras',   # file path (can be .keras or .h5)
    monitor='val_loss',        # metric to monitor
    mode='min',                    # because higher accuracy is better
    save_best_only=True,           # only save when val_accuracy improves
    verbose=1                      # print message when a model is saved
)

In [ ]:
history = transfer_model.fit(
    X_ptb_train_sm_cnn, 
    y_ptb_train_sm,
    epochs=50,
    batch_size=128,
    validation_data=(X_ptb_val_cnn, y_ptb_val),  
    callbacks=[checkpoint]
)

In [ ]:
with open("transfer/cnn6_transfer_lr_bs128_epoch_33_valloss_0.0730.pkl", "wb") as f:
    pickle.dump(history.history, f)


best_model = load_model('transfer/cnn6_transfer_lr_bs128_epoch_33_valloss_0.0730.keras')

test_pred = best_model.predict(X_ptb_test_cnn)
y_test_class = y_ptb_test
y_pred_class = np.argmax(test_pred, axis=1)


print(classification_report(y_test_class, y_pred_class, digits=4))

print(pd.crosstab(y_test_class, y_pred_class, colnames=['Predictions']))


#save results of metrics
with open("transfer/cnn6_transfer_lr_bs128_epoch_33_valloss_0.0730.txt", "w") as file:
    file.write("\nModel: CNN6 transfer MIT \n")
    file.write("\nData augmentation: MIT Smote, PTB None\n")
    file.write("\nConfusion Matrix on test set:\n")
    file.write(str(pd.crosstab(y_test_class, y_pred_class, colnames=['Predictions'])))
    file.write("\n\nClassification Report on test set:\n")
    file.write(classification_report(y_test_class, y_pred_class, digits=4))

In [ ]:
plot_training_history(history, save_dir="transfer", prefix="cnn6_transfer_lr_bs128_epoch_33_valloss_0.0730")